## Controlling webcam

In [1]:
import numpy as np
import cv2 as cv
from PIL import Image
import ipywidgets
from IPython.display import clear_output

In [2]:
from io import BytesIO
import IPython.display
import PIL.Image

def get_source(id):
    return (
        "nvarguscamerasrc sensor-id=%d sensor-mode=%d ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            id, # camera id
            3, # mode
            640, # width
            480, # height
            10, # frame rate
            0, # flip method
            640, # display width
            480, # display height
        )
    )

def camera_capture(id):
    camera = cv.VideoCapture(get_source(id))
    camera.set(3, 640)
    camera.set(4, 480)
    ret, image = camera.read()

    if not ret:
        return None

    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    camera.release()
    return image

def camera_stream(id):
    camera = cv.VideoCapture(get_source(id))
    camera.set(3, 640)
    camera.set(4, 480)
    try: 
        while True:
            ret, frame = camera.read()
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            
            stream = BytesIO()
            PIL.Image.fromarray(frame).save(stream, format="jpeg")
            IPython.display.display(IPython.display.Image(data=stream.getvalue()))

            IPython.display.clear_output(wait=True)
    except KeyboardInterrupt:
        camera.release()
        print("Stream Stopped")
        
def camera_stream2():
    camera0 = cv.VideoCapture(get_source(0))
    camera0.set(3, 640)
    camera0.set(4, 480)
    
    camera1 = cv.VideoCapture(get_source(1))
    camera1.set(3, 640)
    camera1.set(4, 480)
    try: 
        while True:
            ret1, frame1 = camera1.read()
            ret0, frame0 = camera0.read()
            frame0 = cv.cvtColor(frame0, cv.COLOR_BGR2GRAY)
            frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
            
            frame = np.hstack((frame0, frame1))
            
            stream = BytesIO()
            PIL.Image.fromarray(frame).save(stream, format="jpeg")
    
            IPython.display.display(IPython.display.Image(data=stream.getvalue()))
            IPython.display.clear_output(wait=True)
    except KeyboardInterrupt:
        camera0.release()
        camera1.release()
        print("Stream Stopped")

In [10]:
camera_stream2()

Stream Stopped


## Depth Map

In [3]:
ID_LEFT = 0
ID_RIGHT = 1

left, right = camera_capture(ID_LEFT), camera_capture(ID_RIGHT)